# Production

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его сущесвтенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
- А точно нужно сортировать по вероятности?
- Какую метрику использовать?
- Сколько раз в неделю отпрпавляем рассылку?
- В какое время отправляем рассылку?
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [2]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [3]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
#departments=item_features.groupby('department')['department'].count().sort_values(ascending=False)
#df_department=pd.DataFrame(data=departments)
#df_department['percent']=df_department['department']/df_department['department'].sum()*100
#TOP_department=df_department.loc[df_department['percent']<0.5].index.tolist()
#notpopulat_items=item_features[~item_features['department'].isin(TOP_department)].item_id.tolist()
#data=data[data['item_id'].isin(notpopulat_items)]
#data

In [5]:
def prefilter_items(data, data_items):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
    popularity['user_id']=popularity['user_id']/data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    one_year_items=data.loc[data['week_no']<53].item_id.unique().tolist() #Список товаров проданных за последний год
    data = data[data['item_id'].isin(one_year_items)]
    
    # Уберем не интересные для рекоммендаций категории (department)
    data=data.merge(data_items[['item_id','department']], how='left', on=['item_id'])
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1)) #сразу добавим и цену !
    departments=data.groupby('department')['department'].count().sort_values(ascending=False)
    df_department=pd.DataFrame(data=departments)
    df_department['percent']=df_department['department']/df_department['department'].sum()*100

    TOP_department=df_department.loc[df_department['percent']>0.1].index.tolist() # выбираем категории, где покупок было >0.1%
    data=data[data['department'].isin(TOP_department)]
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    smoll_price=data.loc[data['price']<data['price'].quantile(0.2)].item_id.unique().tolist() #quantile(0.2)=1
    data=data[~data['item_id'].isin(smoll_price)]
        
    # Уберем слишком дорогие товары
    big_price=data.loc[data['price']>data['price'].quantile(0.99995)].item_id.unique().tolist() #quantile(0.99995)=36.85
    data=data[~data['item_id'].isin(big_price)]
    
    return data 
    
def postfilter_items(user_id, recommednations):
    pass

In [6]:
prefilter_items(data_train, item_features)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,department,price
5,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0,GROCERY,2.99
6,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0,GROCERY,1.89
8,1364,26984896261,1,937406,1,2.50,31742,-0.99,1520,1,0.0,0.0,MEAT-PCKGD,2.50
18,98,26984951769,1,985911,1,1.25,337,-0.34,1937,1,0.0,0.0,GROCERY,1.25
21,1172,26985025264,1,1000493,1,4.44,396,-0.89,946,1,0.0,0.0,DELI,4.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741918,2088,41297771158,635,7442978,1,1.00,304,-0.69,1258,91,0.0,0.0,DRUG GM,1.00
1741922,2088,41297771158,635,13008321,1,2.19,304,0.00,1258,91,0.0,0.0,GROCERY,2.19
1741924,1541,41297771177,635,972931,1,1.99,304,0.00,1300,91,0.0,0.0,GROCERY,1.99
1741928,462,41297773713,635,993339,1,1.99,304,0.00,2040,91,0.0,0.0,GROCERY,1.99


Основные вопросы:

- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)

Пока вариант видится как переодическое переобучение модели. Т.к. акции не так часто мерняют (1 раз в день пусть), то и рек-ая система настраивать/переобучать 1 раз в день

- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)

Возможно использовать несколько моделий. Контентная фильтрация 

- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

Потестить разные цены. Построить график и определить максимальную цены после которой модель дает хуже рез-ты
